In [1]:
from PIL import Image
import csv
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time, os, json
import random
from torch.autograd import Variable
import tqdm
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F

class_names = ['Normal', 'Almost Clear', 'Mild', 'Moderate', 'Severe']

C:\Users\Bae\anaconda3\envs\pytorch\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Bae\anaconda3\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Bae\anaconda3\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(seed=42)

In [3]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, mode = 'Training', transform = None):
        self.root_dir = root_dir
        self.img_path = glob.glob(root_dir + '/*/crop/*.jpg')
        self.transform = transform
        self.mode = mode

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        img = Image.open(self.img_path[idx]).convert('RGB')
        if self.transform:
            img = self.transform(img)
        if self.mode == 'Test':
            return img, self.img_path[idx]
        else:
            dir_name = os.path.dirname(self.img_path[idx])
            file_name = os.path.basename(self.img_path[idx])
            json_full_path = os.path.join(os.path.join(dir_name[:-4], 'metadata'), file_name[:-4] + '.json')
            data = json.load(open(json_full_path))
            if 'iga_grade' in data['annotations'][0]['clinical_info']:
                grade = class_names.index(data['annotations'][0]['clinical_info']['iga_grade'])
            else:
                grade = 0
            return img, grade

In [4]:
class CustomDatasetCropped(Dataset):
    def __init__(self, root_dir, mode = 'Training', transform = None):
        self.root_dir = root_dir
        self.img_path = glob.glob(root_dir + '*.png')
        self.transform = transform
        self.mode = mode
    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        img = Image.open(self.img_path[idx]).convert('RGB')
        
        if self.transform:
            img = self.transform(img)
            
        if self.mode == 'Test':
            return img, self.img_path[idx]
        
        else:
            dir_name = os.path.dirname(self.img_path[idx])
            label = dir_name.split('/')[-1]
            grade = class_names.index(label)

            return img, grade

In [ ]:
train_transforms = transforms.Compose([
#         transforms.Resize(256),
        transforms.RandomResizedCrop(244),
#         transforms.RandomHorizontalFlip(p=0.5),
#         transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(degrees=(0, 180)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

test_transforms = transforms.Compose([
#         transforms.Resize(256),
        transforms.RandomResizedCrop(244),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

# Model

In [ ]:
class Swish(nn.Module):
    def __init__(self):
        super().__init__()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return x * self.sigmoid(x)

## 1. SE Block

In [ ]:
class SEBlock(nn.Module):
    def __init__(self, in_channels, r=4):
        super().__init__()

        self.squeeze = nn.AdaptiveAvgPool2d((1,1))
        self.excitation = nn.Sequential(
            nn.Linear(in_channels, in_channels * r),
            Swish(),
            nn.Linear(in_channels * r, in_channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.squeeze(x)
        x = x.view(x.size(0), -1)
        x = self.excitation(x)
        x = x.view(x.size(0), x.size(1), 1, 1)
        return x

## 2. MB Conv

In [ ]:
class MBConv(nn.Module):
    expand = 6
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, se_scale=4, p=0.5):
        super().__init__()
        # first MBConv is not using stochastic depth
        self.p = torch.tensor(p).float() if (in_channels == out_channels) else torch.tensor(1).float()

        self.residual = nn.Sequential(
            nn.Conv2d(in_channels, in_channels * MBConv.expand, 1, stride=stride, padding=0, bias=False),
            nn.BatchNorm2d(in_channels * MBConv.expand, momentum=0.99, eps=1e-3),
            Swish(),
            nn.Conv2d(in_channels * MBConv.expand, in_channels * MBConv.expand, kernel_size=kernel_size,
                      stride=1, padding=kernel_size//2, bias=False, groups=in_channels*MBConv.expand),
            nn.BatchNorm2d(in_channels * MBConv.expand, momentum=0.99, eps=1e-3),
            Swish()
        )

        self.se = SEBlock(in_channels * MBConv.expand, se_scale)

        self.project = nn.Sequential(
            nn.Conv2d(in_channels*MBConv.expand, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels, momentum=0.99, eps=1e-3)
        )

        self.shortcut = (stride == 1) and (in_channels == out_channels)

    def forward(self, x):
        # stochastic depth
        if self.training:
            if not torch.bernoulli(self.p):
                return x

        x_shortcut = x
        x_residual = self.residual(x)
        x_se = self.se(x_residual)

        x = x_se * x_residual
        x = self.project(x)

        if self.shortcut:
            x= x_shortcut + x

        return x

## 3. Sep Conv

In [5]:
class SepConv(nn.Module):
    expand = 1
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, se_scale=4, p=0.5):
        super().__init__()
        # first SepConv is not using stochastic depth
        self.p = torch.tensor(p).float() if (in_channels == out_channels) else torch.tensor(1).float()

        self.residual = nn.Sequential(
            nn.Conv2d(in_channels * SepConv.expand, in_channels * SepConv.expand, kernel_size=kernel_size,
                      stride=1, padding=kernel_size//2, bias=False, groups=in_channels*SepConv.expand),
            nn.BatchNorm2d(in_channels * SepConv.expand, momentum=0.99, eps=1e-3),
            Swish()
        )

        self.se = SEBlock(in_channels * SepConv.expand, se_scale)

        self.project = nn.Sequential(
            nn.Conv2d(in_channels*SepConv.expand, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels, momentum=0.99, eps=1e-3)
        )

        self.shortcut = (stride == 1) and (in_channels == out_channels)

    def forward(self, x):
        # stochastic depth
        if self.training:
            if not torch.bernoulli(self.p):
                return x

        x_shortcut = x
        x_residual = self.residual(x)
        x_se = self.se(x_residual)

        x = x_se * x_residual
        x = self.project(x)

        if self.shortcut:
            x= x_shortcut + x

        return x

## 4. EfficientNet

In [6]:
class EfficientNet(nn.Module):
    def __init__(self, num_classes=5, width_coef=1., depth_coef=1., scale=1., dropout=0.2, se_scale=4, stochastic_depth=False, p=0.5):
        super().__init__()
        channels = [32, 16, 24, 40, 80, 112, 192, 320, 1280]
        repeats = [1, 2, 2, 3, 3, 4, 1]
        strides = [1, 2, 2, 2, 1, 2, 1]
        kernel_size = [3, 3, 5, 3, 5, 5, 3]
        depth = depth_coef
        width = width_coef

        channels = [int(x*width) for x in channels]
        repeats = [int(x*depth) for x in repeats]

        # stochastic depth
        if stochastic_depth:
            self.p = p
            self.step = (1 - 0.5) / (sum(repeats) - 1)
        else:
            self.p = 1
            self.step = 0


        # efficient net
        self.upsample = nn.Upsample(scale_factor=scale, mode='bilinear', align_corners=False)

        self.stage1 = nn.Sequential(
            nn.Conv2d(3, channels[0],3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(channels[0], momentum=0.99, eps=1e-3)
        )

        self.stage2 = self._make_Block(SepConv, repeats[0], channels[0], channels[1], kernel_size[0], strides[0], se_scale)
        self.stage3 = self._make_Block(MBConv, repeats[1], channels[1], channels[2], kernel_size[1], strides[1], se_scale)
        self.stage4 = self._make_Block(MBConv, repeats[2], channels[2], channels[3], kernel_size[2], strides[2], se_scale)
        self.stage5 = self._make_Block(MBConv, repeats[3], channels[3], channels[4], kernel_size[3], strides[3], se_scale)
        self.stage6 = self._make_Block(MBConv, repeats[4], channels[4], channels[5], kernel_size[4], strides[4], se_scale)
        self.stage7 = self._make_Block(MBConv, repeats[5], channels[5], channels[6], kernel_size[5], strides[5], se_scale)
        self.stage8 = self._make_Block(MBConv, repeats[6], channels[6], channels[7], kernel_size[6], strides[6], se_scale)

        self.stage9 = nn.Sequential(
            nn.Conv2d(channels[7], channels[8], 1, stride=1, bias=False),
            nn.BatchNorm2d(channels[8], momentum=0.99, eps=1e-3),
            Swish()
        ) 

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.dropout = nn.Dropout(p=dropout)
        self.linear = nn.Linear(channels[8], num_classes)

    def forward(self, x):
        x = self.upsample(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.stage5(x)
        x = self.stage6(x)
        x = self.stage7(x)
        x = self.stage8(x)
        x = self.stage9(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.linear(x)
        return x


    def _make_Block(self, block, repeats, in_channels, out_channels, kernel_size, stride, se_scale):
        strides = [stride] + [1] * (repeats - 1)
        layers = []
        for stride in strides:
            layers.append(block(in_channels, out_channels, kernel_size, stride, se_scale, self.p))
            in_channels = out_channels
            self.p -= self.step

        return nn.Sequential(*layers)

In [7]:
def efficientnet_b1(num_classes=5):
    return EfficientNet(num_classes=num_classes,
                        width_coef=1.0,
                        depth_coef=1.1,
                        scale=240/224,
                        dropout=0.2,
                        se_scale=4)

def efficientnet_b2(num_classes=5):
    return EfficientNet(num_classes=num_classes,
                        width_coef=1.1,
                        depth_coef=1.2,
                        scale=260/224.,
                        dropout=0.3,
                        se_scale=4)

def efficientnet_b3(num_classes=5):
    return EfficientNet(num_classes=num_classes,
                        width_coef=1.2,
                        depth_coef=1.4,
                        scale=300/224,
                        dropout=0.3,
                        se_scale=4)

def efficientnet_b4(num_classes=5):
    return EfficientNet(num_classes=num_classes,
                        width_coef=1.4,
                        depth_coef=1.8,
                        scale=380/224,
                        dropout=0.4,
                        se_scale=4)

def efficientnet_b5(num_classes=5):
    return EfficientNet(num_classes=num_classes,
                        width_coef=1.6,
                        depth_coef=2.2,
                        scale=456/224,
                        dropout=0.4,
                        se_scale=4)

def efficientnet_b6(num_classes=5):
    return EfficientNet(num_classes=num_classes,
                        width_coef=1.8,
                        depth_coef=2.6,
                        scale=528/224,
                        dropout=0.5,
                        se_scale=4)

def efficientnet_b7(num_classes=5):
    return EfficientNet(num_classes=num_classes,
                        width_coef=2.0,
                        depth_coef=3.1,
                        scale=600/224,
                        dropout=0.5,
                        se_scale=4)

In [ ]:
# data_dir = 'dataset'

# train_dir = os.path.join(data_dir, 'train')
# valid_dir = os.path.join(data_dir, 'validation')
# test_dir = os.path.join(data_dir, 'test')

# train_dir = os.path.join('DATA_IMAGE_2/trains/*/')
# valid_dir = os.path.join('DATA_IMAGE_2_resize/validation/*/')
# test_dir = os.path.join('DATA_IMAGE_2_resize/test/')

train_dir = os.path.join('DATA_CROPPED_resize/train/*/')
valid_dir = os.path.join('DATA_CROPPED_resize/validation/*/')
test_dir = os.path.join('DATA_CROPPED_resize/test/')

train_dataset = CustomDatasetCropped(train_dir, transform = train_transforms)
valid_dataset = CustomDatasetCropped(valid_dir, transform = test_transforms)
test_dataset = CustomDatasetCropped(test_dir, mode = 'Test', transform = test_transforms)

print(len(train_dataset))
print(len(valid_dataset))
print(len(test_dataset))


# train_dataset = CustomDataset(train_dir, transform = train_transforms)
# valid_dataset = CustomDataset(valid_dir, transform = test_transforms)
# test_dataset = CustomDataset(test_dir, mode = 'Test', transform = test_transforms)

train_loader = DataLoader(train_dataset,
                          batch_size = 32,
                          shuffle = True)
valid_loader = DataLoader(valid_dataset,
                          batch_size = 16,
                          shuffle = True)
test_loader = DataLoader(test_dataset,
                         batch_size = 1,
                         shuffle = False)

# GPU 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#EfficientNet
model_ft = efficientnet_b2()
num_ftrs = 1408
# model_ft.fc = nn.Linear(num_ftrs, 512)
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

# # # Inception
# model_ft = models.efficientnet_b7()
# num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

# print(model_ft)
print("Model loaded")

criterion = nn.CrossEntropyLoss()
# criterion = nn.MultiLabelSoftMarginLoss()

optimizer_ft = optim.Adam(model_ft.parameters(), lr = 0.001)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 5, gamma = 0.85)
# exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=5, eta_min=0.001)
# exp_lr_scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer_ft,
#                                                             T_0=5,
#                                                             T_mult=1,
#                                                             eta_min=0.001)

num_epochs = 100
print("Training start")

# --Model Save--

In [ ]:
def lite_save(state, epoch, save_dir, model):
    os.makedirs(save_dir, exist_ok=True)
    
    target_path = f'{save_dir}/{state}.path.tar'
    
    with open(target_path, "wb") as f:
        torch.save({
            'epoch': epoch,
            'model_state_dict':model.state_dict(),}, f)

In [8]:
MODEL_NAME = 'efficientnet_b2_v1'
SAVE_PATH = f'weights/{MODEL_NAME}/'

## Mixup

In [ ]:
 def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

## Polynomial LRDecay

In [9]:
from torch.optim.lr_scheduler import _LRScheduler

class PolynomialLRDecay(_LRScheduler):
    """Polynomial learning rate decay until step reach to max_decay_step
    
    Args:
        optimizer (Optimizer): Wrapped optimizer.
        max_decay_steps: after this step, we stop decreasing learning rate
        end_learning_rate: scheduler stoping learning rate decay, value of learning rate must be this value
        power: The power of the polynomial.
    """
    
    def __init__(self, optimizer, max_decay_steps, end_learning_rate=0.0001, power=1.0):
        if max_decay_steps <= 1.:
            raise ValueError('max_decay_steps should be greater than 1.')
        self.max_decay_steps = max_decay_steps
        self.end_learning_rate = end_learning_rate
        self.power = power
        self.last_step = 0
        super().__init__(optimizer)
        
    def get_lr(self):
        if self.last_step > self.max_decay_steps:
            return [self.end_learning_rate for _ in self.base_lrs]

        return [(base_lr - self.end_learning_rate) * 
                ((1 - self.last_step / self.max_decay_steps) ** (self.power)) + 
                self.end_learning_rate for base_lr in self.base_lrs]
    
    def step(self, step=None):
        if step is None:
            step = self.last_step + 1
        self.last_step = step if step != 0 else 1
        if self.last_step <= self.max_decay_steps:
            decay_lrs = [(base_lr - self.end_learning_rate) * 
                         ((1 - self.last_step / self.max_decay_steps) ** (self.power)) + 
                         self.end_learning_rate for base_lr in self.base_lrs]
            for param_group, lr in zip(self.optimizer.param_groups, decay_lrs):
                param_group['lr'] = lr

In [ ]:
decay_steps = (len(train_dataset) // 16 + 1) * num_epochs ## Batch _size

plr = PolynomialLRDecay(optimizer_ft,
                        max_decay_steps=decay_steps,
                        end_learning_rate=0.0001,
                        power=9.0)

# Label Smoothing

In [ ]:
def loss_fn(outputs, targets):
    if len(targets.shape) == 1:
        return F.cross_entropy(outputs, targets)
    else:
        return torch.mean(torch.sum(-targets * F.log_softmax(outputs, dim=1), dim=1))

def label_smooth_criterion(outputs, targets, epsilon=0.1):
    num_classes = outputs.shape[1]
    onehot = F.one_hot(targets, num_classes).to(dtype=torch.float, device=device)
    targets = (1 - epsilon) * onehot + torch.ones(onehot.shape).to(
        device
    ) * epsilon / num_classes
    return loss_fn(outputs, targets)

# Train

In [10]:
scaler = torch.cuda.amp.GradScaler()

def train_model(model, criterion, optimizer, scheduler, num_epochs, train_loader, valid_loader):
    best_acc = 0
    model.train()
    train_loss = 0
    train_acc = 0
    train_correct = 0
    train_losses = []
    train_accuracies = []
    valid_losses = []
    valid_accuracies = []

    for epoch in range(num_epochs):
        start = time.time()
        for train_x, train_y in tqdm.tqdm(train_loader, total=len(train_loader)):
            model.train()
           
            train_x, train_y = train_x.to(device), train_y.to(device)
            optimizer.zero_grad()
#             # Mixup
#             train_x, train_y_a, train_y_b, lam = mixup_data(train_x, train_y)
#             train_x, train_y_a, train_y_b = map(Variable, (train_x, train_y_a, train_y_b))
            with torch.cuda.amp.autocast():
                pred = model(train_x)
                _, preds = torch.max(pred, 1)
                loss = criterion(pred, train_y)
#                 loss = mixup_criterion(label_smooth_criterion, pred, train_y_a, train_y_b, lam)
#                 loss.backward()
#                 optimizer.step()
                train_loss += loss.item()
                train_correct += torch.sum(preds == train_y)
            
            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
            scaler.step(optimizer)
            scaler.update()
            
            scheduler.step()

        plr.step()
        
        valid_loss = 0
        valid_acc = 0
        valid_correct = 0

        for valid_x, valid_y in tqdm.tqdm(valid_loader, total=len(valid_loader)):
            with torch.no_grad():
                model.eval()
                valid_x, valid_y = valid_x.to(device), valid_y.to(device)
                pred = model(valid_x)
                loss = criterion(pred, valid_y)
            valid_loss += loss.item()
            pred = model(train_x)
            _, preds = torch.max(pred, 1)
            valid_correct += torch.sum(preds == train_y)
        train_acc = train_correct/len(train_loader.dataset)
        valid_acc = valid_correct/len(valid_loader.dataset)
        
        if valid_acc > best_acc:
            best_acc = valid_acc
            
            lite_save('best', epoch, SAVE_PATH, model)
        lite_save('best', epoch, SAVE_PATH, model)
        
        print(f'{time.time() - start:.3f}sec : [Epoch {epoch+1}/{num_epochs} -> \
              train loss: {train_loss/len(train_loader):.4f},\
              valid loss: {valid_loss/len(valid_loader):.4f}')
        
        train_losses.append(train_loss/len(train_loader))
        train_accuracies.append(train_acc)
        valid_losses.append(valid_loss/len(valid_loader))
        valid_accuracies.append(valid_acc)

        train_loss = 0
        train_acc = 0
        train_correct = 0
    
    return model

C:\Users\Bae\AppData\Roaming\Python\Python38\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


In [ ]:
model_ft = train_model(model_ft,
                       label_smooth_criterion,
                       optimizer_ft,
                       exp_lr_scheduler,
                       num_epochs,
                       train_loader,
                       valid_loader)

In [ ]:
# import ttach as tta

# Five crop_transform
# model_ft = tta.ClassificationTTAWrapper(model_ft, tta.aliases.)

model_ft = models.resnet152()
model_ft.load_state_dict(torch.load('./weights/efficientnet_b7_v1/best.path.tar')['model_state_dict'],strict=False)
model_ft.cuda()
# print('model loaded')

# inference code
f = open('배류나이 배류배류_test_result.csv', 'w', encoding ='utf-8', newline = '')
wr = csv.writer(f)
wr.writerow(['case', 'Predicted Severity', 'Inference time(ms)'])
with torch.no_grad():
    model_ft.eval()
    correct = 0
    losses = 0

    for img, files in test_loader:
        img = img.to(device)
        start = time.time()
        pred = model_ft(img)
        _, preds = torch.max(pred, 1)
        end = time.time()
        preds = preds.cpu().numpy()[0]

        wr.writerow([os.path.basename(files[0])[:-4], class_names[preds], (end - start) * 1000])

f.close()